In [1]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import json

In [9]:
# Функция для подсчета количества страниц на первой странице поиска объявлений о модели
def brand_pages(url):
    
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    
    pages_cnt = page.find_all(class_ = "Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page")
    
    if pages_cnt != []:
        last_tag = pages_cnt[-1]
        cnt_text = last_tag.find(class_ = "Button__text").text
        return int(cnt_text)
    else:
        return 1

In [3]:
# Функция для извлечения всех ссылок на автомобили на текущей странице очередной модели
def get_auto_links(url):
    
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    
    links = []
    hrefs = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for href in hrefs:
        links.append(href.get("href"))
    
    return links

In [4]:
# Функция по извлечению данных со страницы автомобиля в словарь auto
def get_auto_data(url):
    
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    
    # Будем заполнять сллварь, так как его потом легко превратить в датафрейм
    auto = {}  
    auto['car_url'] = url
    auto['parsing_unixtime'] = int(time.time())
    
    try:
        for script in page.find_all("script"):
            try:
                if 'complectation":{"id"' in str(script):
                    scr = str(script)  
                    auto['complectation_dict'] = re.search(r'complectation":{"id.*?}', scr)[0][15:]
            except:
                auto['complectation'] = None

            try:
                if 'equipment":{' in str(script):
                    scr = str(script) 
                    auto['equipment_dict'] = re.search(r'equipment":{.*?}', scr)[0][11:]
            except:
                auto['equipment_dict'] = None

            try:
                if '"model_info":' in str(script):
                    scr = str(script)  
                    auto['model_info'] = re.search(r'"model_info":{.*?}', scr)[0][13:]
                    auto['model_name'] = re.search(r'model_info":{"code":".*?"', scr)[0][20:].strip('"')
            except:
                auto['model_info'] = None
                auto['model_name'] = None        
            try:
                if 'super_gen":{' in str(script):
                    scr = str(script)  
                    auto['super_gen'] = re.search(r'super_gen":{.*?}', scr)[0][11:]
            except:
                auto['equipment_dict'] = None        
            try:       
                if 'vendor":"' in str(script):
                    scr = str(script)  
                    auto['vendor'] = re.search(r'vendor":".*?"', scr)[0][9:].strip('"')
            except:
                auto['vendor'] = None        

        for tag in page.find_all('div'):
            if tag.get("title") == "Идентификатор объявления":
                auto['sell_id'] = re.search(r'\d+', tag.text)[0]


        Breadcrumbs = page.find_all('a', class_="Link Link_color_gray CardBreadcrumbs__itemText")
        auto['brand'] = Breadcrumbs[0].text
        auto['model_name'] = Breadcrumbs[1].text
        auto['bodyType'] = Breadcrumbs[3].text


        tags = page.find_all('li', class_="CardInfoRow CardInfoRow_year")
        auto['productionDate'] = tags[0].text[-4:]

        tags = page.find_all('li', class_="CardInfoRow CardInfoRow_color")
        auto['color'] = tags[0].text[4:]

        tags = page.find_all('li', class_="CardInfoRow CardInfoRow_engine")
        txt = tags[0].text.replace(' ','').replace('\xa0л.с.','').split('/')
        auto['engineDisplacement'] = txt[0][9:12]
        auto['enginePower'] = txt[1]
        auto['fuelType'] = txt[2]

        tags = page.find_all('li', class_="CardInfoRow CardInfoRow_transmission")
        txt = str(tags)
        st = txt.rpartition('<span class="CardInfoRow__cell">')[2].partition('</span></li>]')
        auto['vehicleTransmission'] = st[0]

        tags = page.find_all('li', class_="CardInfoRow CardInfoRow_kmAge")
        txt = str(tags)
        st = txt.rpartition('<span class="CardInfoRow__cell">')[2].partition('</span></li>]')
        auto['mileage'] = st[0].replace('\xa0','').replace('км','')

        try:
            tags = page.find_all('div', class_="CardDescription__textInner")
            txt = str(tags)
            st = txt.partition('<span>')[2].partition('</span>')
            auto['description'] = st[0].replace('<br/>',' ')
        except:
            auto['description'] = None 


        tags = page.find_all('span', {'class': 'OfferPriceCaption__price'})
        st = tags[0].text.replace('\xa0','')
        auto['priceCurrency'] = st[-1]
        auto['price'] = st[:-1]

        tags = page.find_all('a', {'class': 'Link Link_color_black'})
        auto['productionDate'] = tags[0].text

        # Часть информации ушла со страницы объявления о продаже автомобиля
        # За пропавшей информацией перейдем на страницу модели
        tags = page.find_all('a', {'class': 'Link SpoilerLink CardCatalogLink SpoilerLink_type_default'})
        link_model = tags[0].get("href")

        tags = page.find_all('a', {'class': 'Link SpoilerLink CardCatalogLink SpoilerLink_type_default'})
        link = tags[0].get("href")
        responseM = requests.get(link, headers={'User-Agent': UserAgent().chrome})
        responseM.encoding ='utf8'
        pageM = BeautifulSoup(responseM.text, 'html.parser')

        tagsM = pageM.find_all('div', {'class': 'search-form-v2-mmm search-accordion i-bem'})
        strM = tagsM[0].get("data-bem")
        j = json.loads(strM)
        ge = j['search-form-v2-mmm']['selectedMmmNames']['generation']
        auto['modelDate'] = ge.replace(' ','').split('–')[0][-4:]

        # Вместо измененного super_gen, в котором теперь лежит совсем другая информация,
        # отдельно вытащим данные, которые некогда лежали в этом поле
        tagsM = pageM.find_all('dt', {'class': 'list-values__label'})
        i = -1
        acc = -1
        nd = -1
        cl =-1
        fr =-1
        for tagM in tagsM:
            i += 1
            if tagM.text=='Разгон до 100 км/ч, с':
                acc = i
            if tagM.text=='Количество дверей':
                nd = i
            if tagM.text=='Клиренс':
                cl = i
            if tagM.text=='Расход топлива, л город/трасса/смешанный':
                fr = i
        
        tagsM = pageM.find_all('dd', {'class': 'list-values__value'})
        auto['numberOfDoors'] = tagsM[nd].text
        
        try:
            auto['acceleration'] = tagsM[acc].text
        except:
            auto['acceleration'] = None
        try:    
            auto['clearance_min'] = tagsM[cl].text
        except:
            auto['clearance_min'] = None
        try:
            auto['fuel_rate'] = tagsM[fr].text
        except:
            auto['fuel_rate'] = None
        

        span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})
        for i,tag in enumerate (span_CardInfoRow__cell):
            try:
                if tag.text == "Владельцы":
                    auto['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            except:
                auto['Владельцы'] = None
            try:        
                if tag.text == "Владение":
                    auto['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            except:
                auto['Владение'] = None
            try:
                if tag.text == "ПТС":
                    auto['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            except:
                auto['ПТС'] = None

            if tag.text == "Привод":
                auto['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            if tag.text == "Руль":
                auto['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 

            try:
                if tag.text == "Состояние":
                    auto['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')
            except:
                auto['Состояние'] = None
            try:
                if tag.text == "Таможня":
                    auto['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            except:
                auto['Таможня'] = None
    except:
        pass
    
    return auto

Скачивались только те марки, которые были в test и в тренировочном наборе, данном по умолчанию.
Русские марки не скачивались, так как они сильно отличаются от иномарок.
Редкие марки не скачивались, тоже чтобы не нарушать статистическую однородность.
Автомобили скачивались только до 2020 года выпуска. Год 2021 не брался, так как это новые автомобили.

Ниже приводится список использованных марок, но скачивание производилось не для всех сразу, а порциями в несколько дней. 
Для этого список разбивался на части.

In [11]:
# Список марок автомобилей
# brands = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
#            'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI',
#            'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER',
#            'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'HYUNDAI',
#            'JAGUAR', 'JEEP', 'KIA', 'MAZDA', 'MINI',
#            'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT',
#            'SUBARU', 'SUZUKI', 'GREAT_WALL', 'LAND_ROVER', 'SSANG_YONG']

brands = ['PONTIAC','MINI']  

df = pd.DataFrame()       # датафрейм для закачки данных одной марки автомобиля

for brand in brands:
    url_brand_list = []   # список ссылок на отдельные страницы данных одной марки автомобиля
    
    url = 'https://auto.ru/moskva/cars/' + brand.lower() + '/used/?year_to=2020'
    max_page = brand_pages(url) + 1
        
    for page in range(1, max_page):
        url = url + '&page=' + str(page)
        url_brand_list.append(url)

    # Получаем список списков ссылок на объявления о всех автомобилях текущей марки со всех страниц
    page_list = [] 
    try:
        page_list = Parallel(n_jobs = 2)(delayed(get_auto_links)(url) for url in url_brand_list)
    except:
        pass 

    # Информируем, сколько страниц данной марки надо скачивать
    print(brand + ': ' + str(len(page_list)) + ' стр.')
    i = 0
    
    auto_dict_list = [] # список словарей с данными по автомобилям на текущей странице

    # Перебираем отдельные страницы для текущей марки автомобиля
    for links in page_list:
        i += 1
        
        # По каждой ссылке на текущей странице парсим данные и распараллеливаем этот процесс
        try:
            auto_dict_list = Parallel(n_jobs = 2)(delayed(get_auto_data)(url) for url in links)
        except Exception as e:
            print('Ошибка: ' + str(e))
            pass
        
        # Информируем, какая по счету страница скачалась
        print(i)

        # В датасет добавляем очередной автомобиль
        for auto in auto_dict_list:
            df = df.append(auto, ignore_index=True)   
            
    # Сохраняем данные по текущей марке автомобиля
    df.to_csv('carMsk-' + brand + '.csv', index=False)

PONTIAC: 1 стр.
1
MINI: 6 стр.
1
2
3
4
5
6


Собранные датасеты по отдельным маркам впоследствии были слиты в один набор.

Таким же образом были собраны данные и по другим городам, но в обучении модели наличие других городов только ухудшило метрику. 
Поэтому реально использовались данные только из Москвы.